In [7]:
import os
import sys
sys.path.insert(0, "../panopto_dl")
from PanoptoDownloader import PanoptoDownloader

In [15]:
URL = "https://d2y36twrtb17ty.cloudfront.net/sessions/9ff73153-b779-496c-9887-aef6017f9eeb/1d06229d-06cb-4851-a3cb-aef6017f9ef8-8c3f5279-5816-498a-ad5e-af640185811c.mp4"
PATH = "./output.mp4"


def callback(progress: int):
    """
    :param progress: Downloading progress. From 0 to 100
    """
    #print(f"{progress} / 100")

In [16]:
PanoptoDownloader.download(
            URL,
            PATH,
            callback
        )